In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("waqi786/remote-work-and-mental-health")

print("Path to dataset files:", path)

100%|██████████| 93.0k/93.0k [00:00<00:00, 7.70MB/s]

Extracting files...
Path to dataset files: C:\Users\ashle\.cache\kagglehub\datasets\waqi786\remote-work-and-mental-health\versions\1


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\ashle\Project-3\Impact_of_Remote_Work_on_Mental_Health.csv')

In [3]:
null_columns = df.isnull().sum()

In [4]:
null_columns = null_columns[null_columns > 0]
null_columns

Mental_Health_Condition    1196
Physical_Activity          1629
dtype: int64

In [9]:
null_columns.info()

<class 'pandas.core.series.Series'>
Index: 2 entries, Mental_Health_Condition to Physical_Activity
Series name: None
Non-Null Count  Dtype
--------------  -----
2 non-null      int64
dtypes: int64(1)
memory usage: 140.0+ bytes


In [6]:
df_cleaned = df.dropna()

In [16]:
df_cleaned = df.dropna(how='all')

In [17]:
df_cleaned = df.dropna(subset=['Mental_Health_Condition', 'Physical_Activity'])
df_cleaned

,Employee_ID,Age,Gender,Job_Role,Industry,Years_of_Experience,Work_Location,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Stress_Level,Mental_Health_Condition,Access_to_Mental_Health_Resources,Productivity_Change,Social_Isolation_Rating,Satisfaction_with_Remote_Work,Company_Support_for_Remote_Work,Physical_Activity,Sleep_Quality,Region
0,EMP0001,32,Non-binary,HR,Healthcare,13,Hybrid,47,7,2,Medium,Depression,No,Decrease,1,Unsatisfied,1,Weekly,Good,Europe
1,EMP0002,40,Female,Data Scientist,IT,3,Remote,52,4,1,Medium,Anxiety,No,Increase,3,Satisfied,2,Weekly,Good,Asia
6,EMP0007,31,Prefer not to say,Sales,IT,24,Remote,51,7,3,Low,Anxiety,Yes,Decrease,5,Neutral,3,Daily,Poor,Asia
9,EMP0010,30,Female,HR,IT,28,Hybrid,57,6,1,Low,Depression,Yes,Decrease,2,Neutral,1,Weekly,Poor,North America
12,EMP0013,40,Female,Marketing,Consulting,1,Remote,21,7,2,High,Depression,Yes,Decrease,2,Neutral,1,Weekly,Good,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,EMP4992,60,Female,Data Scientist,IT,9,Hybrid,57,4,2,Low,Burnout,Yes,Increase,4,Satisfied,5,Weekly,Average,Oceania
4992,EMP4993,55,Prefer not to say,Project Manager,Consulting,21,Hybrid,45,7,2,High,Anxiety,No,No Change,2,Satisfied,3,Daily,Poor,Africa
4994,EMP4995,40,Prefer not to say,Marketing,IT,17,Hybrid,52,1,2,Medium,Burnout,Yes,Increase,3,Neutral,5,Daily,Good,Oceania
4995,EMP4996,32,Male,Sales,Consulting,4,Onsite,24,2,5,High,Burnout,Yes,Decrease,4,Neutral,1,Weekly,Average,Asia


In [18]:
df_cleaned.to_csv('Clean_Impact_of_Remote_Work_on_Mental_Health.csv', index=False)

In [ ]:
df_cleaned.to_csv('C:/path_to_your_directory/output_file.csv', index=False)